In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

In [2]:
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

/opt/anaconda3/envs/gpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
    )

In [4]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [5]:
examples = [
    {
        "movie": "Avengers: Endgame",
        "answer": "🦸‍♂️🧪🕰️"
     },
     {
        "movie": "The Lion King",
        "answer":"🦁👑🌅"
     },
     {
        "movie": "Top Gun: Maverick",
        "answer": "🛩️👨‍✈️🔥"
     }
]

In [6]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{movie}"),
        ("ai","{answer}"),
    ],
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a Ph.D degree in Movie. And you are insane at movie."),
        MessagesPlaceholder(variable_name="history"),
        fewshot_prompt,
        ("human", "{movie}"),
    ]
)

In [7]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie":movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )

In [8]:
invoke_chain("Moana")


🌊🚣‍♀️🌺

In [9]:
invoke_chain("The Dark Knight")

🦇🃏🌃

In [10]:
invoke_chain("Titanic")

🚢💔🌊

In [11]:
invoke_chain("The Frozen")

❄️👸🎶

In [13]:
invoke_chain("What is the previous answers?")

Previous answers were:
- Moana: 🌊🚣‍♀️🌺
- The Dark Knight: 🦇🃏🌃
- Titanic: 🚢💔🌊
- The Frozen: ❄️👸🎶
- Avengers: Endgame: 🦸‍♂️🧪🕰️
- The Lion King: 🦁👑🌅
- Top Gun: Maverick: 🛩️👨‍✈️🔥The human mentions various movies to the AI, who responds with relevant emojis. The human then asks for a recap of the previous answers, which the AI provides.